In [ ]:
#|hide
import nbdev


In [ ]:
#|export
from __future__ import annotations
from typing import Callable, TypeVar,  Generic, Union, Optional, Set, Protocol, Any
from typing_extensions import Annotated
from fastcore.test import test_eq, test, test_fail
from fastcore.basics import patch

In [ ]:
#| default_exp stores

# Stores
> A Svelte stores implementation in Python.  

See the orginal [Svelte Stores](https://svelte.dev/docs#run-time-svelte-store).

## The Svelte Store contract

1. A store must contain a `.subscribe` method, which must accept as its argument a `subscription function`(aka Subscriber or Callback). This `subscription function` must be immediately and synchronously called with the store's current value upon calling `subscribe`. All of a store's active subscription functions must later be synchronously called whenever the store's value changes.

1. The `.subscribe` method must return an `unsubscribe function`(aka Unsubscriber). Calling an `unsubscribe function` must `stop` its subscription, and its corresponding `subscription function` must not be called again by the store.

1. A store may optionally contain a `.set` method, which must accept as its argument a new value for the store, and which synchronously calls all of the store's active subscription functions. Such a store is called a writable store.


~~For interoperability with RxJS Observables, the .subscribe method is also allowed to return an object with an .unsubscribe method, rather than return the unsubscription function directly. Note however that unless .subscribe synchronously calls the subscription (which is not required by the Observable spec), Svelte will see the value of the store as undefined until it does.~~

[Store Contract Documentation](https://svelte.dev/docs#component-format-script-4-prefix-stores-with-$-to-access-their-values-store-contract)


#### Types Definition

In [ ]:
#| exports

T = TypeVar("T")
covT = TypeVar("covT", covariant=True)
Subscriber = Callable[[T], None] # a callback
Unsubscriber = Callable[[], None] # a callback to be used upon termination of the subscription    
Updater = Callable[[T], T]

In [ ]:
#| exports

class StoreProtocol(Protocol, Generic[covT]):
    ''' The Svelte Store ~~contract~~ protocol. '''
    def subscribe(self, subscriber: Subscriber[T]) -> Unsubscriber: ...

In [ ]:
#| exporti

class Store(StoreProtocol[T]):
    ''' A base class for all stores.'''
    value: T
    subscribers: Set[Subscriber]
    def __init__(self, /, **kwargs): 
        self.__dict__.update(kwargs) # see SimpleNamespace: https://docs.python.org/3/library/types.html
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.get()!r})"
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return lambda: None
    def get(self) -> T: return self.value

class Readable(Store[T]): pass

class Writable(Store[T]):
    set: Subscriber
    update: Optional[Callable[[Updater],None]] = None

## Writable Stores

Let's start with a simple implementation of a writable store:

In [ ]:
#|hide
#|export
from sveltish.util import safe_not_equal

In [ ]:
#| exports
class Writable(Store[T]):
    ''' A Writable Store.'''
    def __init__(self, 
                 initial_value: T = None # the initial value of the store
                 ) -> None:
        self.value: T = initial_value
        self.subscribers: Set[Subscriber] = set() # callbacks to be called when the value changes

    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        self.subscribers.add(callback)
        callback(self.value)

        def unsubscribe() -> None:
            # the unsubscribe can be called multiple times, 
            # so we need to check if the callback is still in the set
            self.subscribers.remove(callback) if callback in self.subscribers else None
        return unsubscribe
    
    def set(self, new_value: T) -> None:
        if (safe_not_equal(self.value, new_value)):
            self.value = new_value
            for subscriber in self.subscribers:
                subscriber(new_value)
                
    def update(self, fn: Callable[[T], T]) -> None:
        self.set(fn(self.value))
    
    def __len__(self) -> int:
        ''' The length of the store is the number of subscribers.'''
        return len(self.subscribers)

#### A `Writable Store` in action

Let's create a `Writable`.  Remember that it returns an unsubscriber function.

In [ ]:
count = Writable(0)
values = []
unsubscribe = count.subscribe(lambda x: values.append(x))
count, values, type(unsubscribe)

(Writable(0), [0], function)

A __Writable__ can be set from the outside. When it happens, all its subscribers will react.

In [ ]:
type(count)

__main__.Writable

In [ ]:
count.set(1)
count.update(lambda x: x+1)
count, values

(Writable(2), [0, 1, 2])

After unsubscribing, the subscriber should not be called anymore:

In [ ]:
unsubscribe()
count.set(3)
count.update(lambda x: x+1)
count, values

(Writable(4), [0, 1, 2])

In [ ]:
#| hide
test_eq(values, [0,1,2])


You can create an empty `Writable Store`.

In [ ]:
store = Writable()
values = []
unsubscribe = store.subscribe(lambda x: values.append(x))
unsubscribe()
store, values

(Writable(None), [None])

In [ ]:
#| hide
test_eq(values, [None])

If you try to unsubscibe twice, it won't break.  It just does nothing the second time.

In [ ]:
unsubscribe()

Stores assume mutable objects. 

::: {.callout-note}
In Python everythong is an object.  Here we are calling an object something that is not a primitive (eg. int, bool, etc)
:::

In [ ]:
class Bunch:
    __init__ = lambda self, **kw: setattr(self, '__dict__', kw)

obj = Bunch()
called = 0
store = Writable(obj)
def callback(x):
    global called
    called += 1
store.subscribe(callback)
obj.a = 1 #type: ignore
store.set(obj)
store, called

(Writable(<__main__.Bunch object>), 2)

In [ ]:
#|hide
test_eq(called, 2)

## Readable Stores


A `Readable Store` is a `Writable Store` with protected `set` and `update` methods.

In [ ]:
class Readable(Writable): 
    ''' A Readable Store.'''
    def set(self, *args, **kwargs): raise Exception("Cannot set a Readable Store.")
    def update(self, *args, **kwargs): raise Exception("Cannot update a Readable Store.")

In [ ]:
b = Readable(10)

In [ ]:
u = b.subscribe(lambda x: print("2:",x))

2: 10


We can subscribe to our `readable`, but nothing happens, we cannot `set` a `Readable` from the outside.

In [ ]:
try:
    b.set("bar") # should fail
except Exception as error:
  print(error)


Cannot set a Readable Store.


In [ ]:
#| hide
test_fail(lambda: b.set("bar"), contains="Cannot set a Readable Store.")

You also can create an empty `Readable Store`. 

In [ ]:
store = Readable()
values = []
unsubscribe = store.subscribe(lambda x: values.append(x))
unsubscribe()
store, values

(Readable(None), [None])

In [ ]:
#| hide
test_eq(values, [None])

A `store` that does not change is not useful. A `Readable` is like a `writable` where there is only one "thing" from the outside that can change its value. Lets change `writable` to add this "thing", which we will call a `Notifier`.

#### Refactoring the Writable Store

In [ ]:
#|export
Notifier = Callable[[Subscriber], Union[Unsubscriber, None]]

In [ ]:
#|exports

@patch 
def __init__(self:Writable,
                initial_value: Any = None, # The initial value of the store
                start: Notifier = lambda x: None # A Notifier (Optional)
                ) -> None:
    self.value = initial_value
    self.subscribers: Set[Subscriber] = set() #type: ignore
    self.stop: Optional[Unsubscriber] = None  #type: ignore
    self.start: Notifier = start              #type: ignore


@patch
def subscribe(self:Writable, callback: Subscriber) -> Unsubscriber:
    self.subscribers.add(callback)
    if (len(self.subscribers) == 1):
        self.stop = self.start(callback) or (lambda: None) #type: ignore
    callback(self.value)

    def unsubscribe() -> None:
        self.subscribers.remove(callback) if callback in self.subscribers else None
        if (len(self.subscribers) == 0):
            self.stop() if self.stop else None #type: ignore
            self.stop = None #type: ignore
    return unsubscribe

In [ ]:
# #|exports
# class Writable(Store[T]):
#     ''' A Writable Store.'''
#     def __init__(self:Writable[T], 
#                  initial_value: T=None, # The initial value of the store
#                  start: Notifier=lambda x: None # A Notifier (Optional)
#                  ) -> None:
#         self.value: T = initial_value
#         self.subscribers: Set[Subscriber] = set()
#         self.stop: Optional[Unsubscriber] = None
#         self.start: Notifier = start

#     def subscribe(self, callback: Subscriber) -> Unsubscriber:
#         self.subscribers.add(callback)
#         if (len(self.subscribers) == 1):
#             self.stop = self.start(callback) or (lambda: None)
#         callback(self.value)

#         def unsubscribe() -> None:
#             self.subscribers.remove(callback) if callback in self.subscribers else None
#             if (len(self.subscribers) == 0):
#                 self.stop() if self.stop else None
#                 self.stop = None
#         return unsubscribe
    
#     def set(self, new_value: T) -> None:
#         if (safe_not_equal(self.value, new_value)):
#             self.value = new_value
#             for subscriber in self.subscribers:
#                 subscriber(new_value)
    
#     def update(self, fn: Callable[[T], T]) -> None:
#         self.set(fn(self.value))
    
#     def __len__(self) -> int:
#         return len(self.subscribers)

In [ ]:
#| hide
# previous tests shoudn't fail
count = Writable(0)
values = []
unsubscribe = count.subscribe(lambda x: values.append(x))
count.set(1)
count.update(lambda x: x+1)
unsubscribe()
count.set(3)
count.update(lambda x: x+1)
test_eq(values, [0,1,2])
store = Writable()
values = []
unsubscribe = store.subscribe(lambda x: values.append(x))
unsubscribe()
test_eq(values, [None])
unsubscribe()
test_eq(unsubscribe(), None)
obj = Bunch()
called = 0
store = Writable(obj)
def callback(x):
    global called
    called += 1
store.subscribe(callback)
obj.a = 1 #type: ignore
store.set(obj)
test_eq(called, 2)

But now, we can start the store with a `Notifier` that asynchronously set the value of the store from the outside.

Let's test by creating an asynchronous notifier.

In [ ]:
from threading import Event, Thread
import time

In [ ]:
def every(interval, func, *args):
    stopped = Event()
    def loop():
        while not stopped.wait(interval): # the first call is in `interval` secs
            func(*args)
    Thread(target=loop).start()    
    return stopped.set

def start(set): # notifier
    count = 0
    def incrementCounter():
        nonlocal count
        count = count +1
        set(count)
    cancel = every(1, incrementCounter)
    return cancel

def myset(x):
    value = x
    print("myset:", value)

In [ ]:
stop = start(myset)
time.sleep(3)
stop()

myset: 1
myset: 2
myset: 3


In [ ]:
b = Writable(0, start)

In [ ]:
b

Writable(0)

In [ ]:
u1 = b.subscribe(lambda x: print("1:",x))

1: 0


In [ ]:
time.sleep(4)
u1()

1: 1
1: 2
1: 3
1: 4


In [ ]:
#|hide
# calls provided subscribe handler
called = 0
def callback(x):
    global called
    called += 1
    def unsubscribe():
        global called
        called -= 1
    return unsubscribe
store = Writable(0, callback)
unsubscribe1 = store.subscribe(lambda x:None)
test_eq(called, 1)
unsubscribe2 = store.subscribe(lambda x:None)
test_eq(called,1)
unsubscribe1()
test_eq(called,1)
unsubscribe2()
test_eq(called,0)

In [ ]:
#|hide
# only calls subscriber once initially, including on resubscriptions
num = 0
def start(set):
    global num
    num+=1
store = Writable(0, start)
count1, count2 = 0,0
def callback1(x):
    global count1
    count1+=1
def callback2(x):
    global count2
    count2+=1
store.subscribe(callback1)()
test_eq(count1, 1)
unsubscribe = store.subscribe(callback2)
test_eq(count2, 1)
unsubscribe()

Nice, it works. Now, let's fix Readable.

#### Refactoring the Readable Store

In [ ]:
#|exports
class Readable(Writable[T]):
    ''' A Readable Store.''' 
    def __init__(self, 
                 initial_value: T, # The initial value of the store
                 start: Notifier # A Notifier 
                ) -> None:
        super().__init__(initial_value, start)
    def set(self, *args, **kwargs): raise Exception("Cannot set a Readable Store.")
    def update(self, *args, **kwargs): raise Exception("Cannot update a Readable Store.")

### A `Readable` in action

Now, we need to provide a `Notifier` to create a `Readable` store:

In [ ]:
try:
    c = Readable(0) # shoud fail
except Exception as error:
    print(error)

__init__() missing 1 required positional argument: 'start'



::: {.callout-note} The Svelte Store api allow you to create a Readable Store without a Notifier. See discussion [here.](https://github.com/sveltejs/svelte/issues/8300)

In [ ]:
#|hide
test_fail(lambda: Readable(0))

In [ ]:
c = Readable(0, start)
c

Readable(0)

Notice that while there is no subscribers, the `Notifier` is not started.

In [ ]:
time.sleep(3)
c

Readable(0)

The first subscriber starts the `Notifier`.

In [ ]:
stop = c.subscribe(lambda x: print("1:",x))

1: 0


In [ ]:
time.sleep(3)
stop()

## Derived Stores

A `Derived Store` takes a store and transforms it into another store.

In [ ]:
#|exports
class Derived(Writable):
    ''' A Derived Store.'''
    def __init__(self,
                  source: Store, # The source store
                  fn: Updater # A function that takes the source store's value and returns a new value
                  ) -> None:
        self.target = Writable(source.get())
        self.start: Notifier = lambda x: self.target.set(fn(x))
        self.stop = source.subscribe(self.start)
    def get(self) -> T: return self.target.get()
    def set(self, *args, **kwargs): raise Exception("Cannot set a Derived Store.")
    def update(self, *args, **kwargs): raise Exception("Cannot update a Derived Store.")
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return self.target.subscribe(callback)

### A `Derived Store` in action

In [ ]:
a = Writable(1)
u1 = a.subscribe(lambda x: print("1:",x))

1: 1


In [ ]:
b = Derived(a, lambda x: x*2)

In [ ]:
a,b

(Writable(1), Derived(2))

In [ ]:
a, b.get()

(Writable(1), 2)

In [ ]:
#|hide
test_eq(b.get(), 2)

In [ ]:
a.set(2), a,b

1: 2


(None, Writable(2), Derived(4))

In [ ]:
#|hide
test_eq(b.get(), 4)

In [ ]:
u1 = b.subscribe(lambda x: print("2:",x))

2: 4


In [ ]:
a.set(42)

2: 84
1: 42


In [ ]:
user = Writable({"name": "John", "age": 42})
user

Writable({'name': 'John', 'age': 42})

In [ ]:
user.update(lambda x: x | {"age": 21})
user

Writable({'name': 'John', 'age': 21})

In [ ]:
u1 = user.subscribe(lambda x: print("user_subscriber_1:",x))

user_subscriber_1: {'name': 'John', 'age': 21}


In [ ]:
user.update(lambda x: x | {"age": 42})

user_subscriber_1: {'name': 'John', 'age': 42}


In [ ]:
name = Derived(user, lambda x: x["name"])
name

Derived('John')

In [ ]:
u2 = name.subscribe(lambda x: print("name_subscriber_1:",x))

name_subscriber_1: John


In [ ]:
user.update(lambda x: x | {"age": 56})

user_subscriber_1: {'name': 'John', 'age': 56}


In [ ]:
user.update(lambda x: x | {"name": "Fred"})

name_subscriber_1: Fred
user_subscriber_1: {'name': 'Fred', 'age': 56}


Svelte Derived Stores allow us to use derive from several source stores, let's refactor.

#### Refactoring Derived Stores

In [ ]:
#|exports
@patch # does not like type annotations
def __init__(self:Derived, 
             s: Union[Store, list[Store]],
             fn: Callable,
             ) -> None:
    isStore = isinstance(s, Store)
    isList = isinstance(s, list) and all([isinstance(x, Store) for x in s])
    if not isStore and not isList: raise Exception("s must be a Store or a list of Stores")
    self.sources:list[Store] = [s] if isStore else s # type: ignore
    self.fn = fn # type: ignore
    self.target = Writable(None)
    self.unsubscribers = [(lambda s=s: s.subscribe(self._update))(s) for s in self.sources] # type: ignore

@patch
def _update(self:Derived, x): # ignore the new value and just refresh the target from sources
    values = [(lambda s=s: s.get())(s) for s in self.sources] # type: ignore
    self.target.set(self.fn(*values)) # type: ignore

In [ ]:
a:Store = Writable('foo')
b = Writable('bar')
d = Derived([a,b], lambda a,b: f"{a}_{b}") # type: ignore

In [ ]:
test_eq(d.get(), "foo_bar")

In [ ]:
u1 = d.subscribe(lambda x: print("d1:",x))

d1: foo_bar


In [ ]:
a.set('fonzie')

d1: fonzie_bar


In [ ]:
test_eq(d.get(), "fonzie_bar")

In [ ]:
b.set('bach')

d1: fonzie_bach


In [ ]:
test_eq(d.get(), "fonzie_bach")

In [ ]:
#|hide
nbdev.nbdev_export()